In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
try:
    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=CLUSTER_URL,
        auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
        headers={
            "X-OpenAI-Api-Key": OPENAI_API_KEY,
            "X-Cohere-Api-Key": COHERE_API_KEY
        },
        additional_config=AdditionalConfig(
            timeout=Timeout(init=60, query=240, insert=480)
        )
    )

    ready = client.is_ready()
    server_version = client.get_meta()["version"]
    client_version = weaviate.__version__
    live = client.is_live()
    connected = client.is_connected()

    print(f"Weaviate Ready: {ready}")
    print(f"Weaviate Client Version: {client_version}")
    print(f"Weaviate Server Version: {server_version}")
    print(f"Weaviate Live: {client.is_live()}")
    print(f"Client Connected: {connected}")

except Exception as e:
    print(f"Error connecting to Weaviate: {e}")


In [ ]:
from weaviate.classes.config import Configure
from weaviate.classes.config import Property, DataType, Tokenization

client.collections.create(
    name="bringvectorssexamlpe",
    vectorizer_config=Configure.Vectorizer.none(),
    inverted_index_config=Configure.inverted_index(
        index_timestamps = True
    ),
    replication_config=Configure.replication(factor=3, async_enabled=True),
    properties=[
        Property(
            name="title",
            data_type=DataType.TEXT,
            tokenization=Tokenization.WORD,
            skip_vectorization=False
        ),
        Property(
            name="body",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD,
            skip_vectorization=False
        )
    ]
)

In [ ]:
# Create Objects with some vectors
col = client.collections.get("bringvectorssexamlpe")
uuid = col.data.insert(
    properties={
        "title": "This vector DB is OSS and supports automatic property type inference on import",
        "body": "Weaviate",
    },
    vector=[0.12345] * 1536
)
print(uuid)



fb7cc5a5-da08-4b65-b60f-253e6bf30c6c


In [ ]:
# Do some near vector search
response = col.query.near_vector(
    near_vector=[0.12345] * 1536,
    limit=2,
)